<a href="https://colab.research.google.com/github/thiagoudweb/Projeto-RAg---1a-VA/blob/main/RAGIPOB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-groq
!pip install streamlit streamlit_chat
!pip install sentence-transformers
!pip install chromadb
!pip install pypdf unstructured
!pip install PyPDF2
!pip install streamlit




In [2]:
import langchain
import streamlit as st
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [5]:
import PyPDF2

def carregar_texto_pdf(caminho_pdf):
    texto_completo = ""
    with open(caminho_pdf, "rb") as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        for pagina in range(len(leitor.pages)):
            texto_completo += leitor.pages[pagina].extract_text() + "\n"
    return texto_completo


In [6]:
from google.colab import files

uploaded = files.upload()  #upload do arquivo manual
caminho_pdf = list(uploaded.keys())[0]

texto_prospecto = carregar_texto_pdf(caminho_pdf)
print(texto_prospecto[:1000])


Saving book.pdf to book (1).pdf
As informações contidas neste Prospecto Preliminar estão sob análise da Comissão de Valores Mobiliários, a qual ainda não se man ifestou a seu respeito. O presente Prospecto Preliminar está sujeito à complementação e correção. O Prospecto Definitivo estará  disponível nas páginas da rede mundial de computadores da Companhia; das
Instituições Participantes da Oferta; das entidades administradoras de mercado organizado de valores mobiliários onde os valore s mobiliários da Companhia sejam admitidos à negociação; e da CVM.  ESTE DOCUMENTO É UMA MINUTA INICIAL SUJEITA A ALTERAÇÕES E COMPLEMENTAÇÕES, TENDO SIDO ARQUIVADO NA COMISSÃO DE VALORES MOBILIÁ RIOS PARA FINS EXCLUSIVOS DE ANÁLISE E EXIGÊNCIAS 
POR PARTE DESSA AUTARQUIA. ESTE DOCUMENTO, PO RTANTO, NÃO SE CARACTERIZA COMO O PROSPECTO PRELIMINAR DA OFERTA E NÃO CONSTITUI U MA OFERTA DE VENDA OU UMA SOLICITAÇÃO PARA 
OFERTA DE COMPRA DE TÍTULOS E VALORES MOBILIÁRIOS NO BRASIL, NOS ESTADOS UNIDOS DA AMÉRIC

In [11]:
def dividir_texto(texto, tamanho_chunk=500, sobreposicao=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamanho_chunk,
        chunk_overlap=sobreposicao,
        separators=["\n\n", "\n", " ", ""]
    )
    return splitter.split_text(texto)




In [17]:
 chunks = dividir_texto(texto_prospecto)

In [14]:
# modelo de embeddings usando lib hugging Face
modelo_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# criar embeddings para os chunk de textos
vetores_texto = [modelo_embedding.embed_query(chunk) for chunk in chunks]



In [15]:
# conectando banco para guardar os vetores
chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=modelo_embedding)

# Aadd textos e embeddinss ao banco
ids = [f"chunk_{i}" for i in range(len(chunks))]  # criando identificação para chunk
chroma_db.add_texts(texts=chunks, ids=ids)

print("Base de dados vetorial criada e populada com sucesso!")


<ipython-input-15-28bb1d62f53d>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=modelo_embedding)


Base de dados vetorial criada e populada com sucesso!
